In [1]:
%load_ext autoreload
%autoreload 2
import lusee
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 3.5]
matplotlib.rcParams['figure.dpi'] = 100
from scipy.ndimage import gaussian_filter
from scipy.interpolate import interp1d
from scipy.integrate import simpson
import refspec
import os
import tqdm.notebook as tqdm

In [2]:
cfg = refspec.SpecConfig()
cfghelp = refspec.SpecConfigHelper(cfg)
sampling_rate = cfg.sampling_rate
block_size = cfg.Nfft ## irrelevant here
Nsec = 0.1
Nsamples = int(Nsec*sampling_rate)
Nblocks_gen = Nsamples//block_size+2*cfg.Ntaps
Nchannels = 1

cfg.zoomin_st    = 10
cfg.zoomin_en    = 11; # 500 kHz times 4 / 100kHz
Nzoom = 64
Nzoomtaps = 1
cfghelp.set_zoom(Nzoom,Nzoomtaps,None)

cfg.Nchannels       = 1
cfg.Average1Size    = 128
cfg.Average2Size    = 1 
assert(cfg.Average2Size>0)
fundamental         = cfg.fundamental_frequency()
outf = np.linspace (-50,50,2000)
for Ntaps in [4,8]:
    out = []
    outz = []
    cfg.Ntaps        =  Ntaps
    for df in tqdm.tqdm(outf):
        tone = 250+df
        source = refspec.SignalGenerator(block_size, Nchannels, Nblocks_gen, tone*1e3, sampling_rate, 100.,0.)
        output = refspec.SpecOutput(cfg)
        spectrometer = refspec.RefSpectrometer(source, cfg)
        output = refspec.SpecOutput(cfg)
        spectrometer.run(output)
        out.append(output.get_avg_pspec(0,10))
        outz.append(np.array(output.get_avg_pspec_zoom(0)))
    out = np.array(out)
    outz = np.array(outz)
    np.savetxt(f'zoom_response_{cfg.Ntaps}tap.txt',np.vstack([[outf],[out],outz.T]).T)

  0%|          | 0/2000 [00:00<?, ?it/s]